# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [16]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [2]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [3]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
slim_raw_df.head(3)

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours",{3: 'G G/B ...,"['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),{1: 'Em D C C...,"['canadian pop', 'pop', 'post-teen pop']",100


In [51]:
split_raw_df = slim_raw_df.copy()
splits = []

for row in split_raw_df.chords:
    row_dict = ast.literal_eval(row)
    for key, value in row_dict.items():
        r_or = value.split('|')[0]
        etc1 = r_or.replace('.', ' ')
        ect2 = etc1.replace('*', '')
        p = re.sub(r'[()]', '', ect2)
        s = re.sub(r'^\s+|\s+$', '', p)
        row_dict[key] = re.sub(r'\s+', ' ', s)
    splits.append(row_dict)

split_raw_df.chords = splits
len(splits)

135783

In [52]:
r = random.randint(0, 135782)
print(slim_raw_df.chords[r])
print(split_raw_df.chords[r])

{2: 'B|-------------------3-----------------------------|', 3: 'G|--------2-----------------------4-----2-00h2-----|', 4: 'D|-------------------------------------------------|', 5: 'A|-------------------------------------------------|', 6: 'E|-------------------------------------------------|', 8: 'F#m       E            D', 10: 'F#m                     E          D', 12: 'Am                 G          D', 14: 'Am                      G             D', 17: 'F#m             E                 D', 19: 'F#m  E    D', 29: 'Em7             Em        F     ', 31: 'Em7             Em                        F', 34: 'F', 37: '(Repete Em7, Em, F thing)', 39: 'F#m               E            D', 41: 'F#m         E  D  ', 43: 'F#m               E                 D', 45: 'F#m               E                 D', 47: 'F#m   E   D', 49: 'F#m   E   D'}
{2: 'B', 3: 'G', 4: 'D', 5: 'A', 6: 'E', 8: 'F#m E D', 10: 'F#m E D', 12: 'Am G D', 14: 'Am G D', 17: 'F#m E D', 19: 'F#m E D', 29: 'Em7 Em F', 31: 'Em7 E

#### See Chord Frequency

In [53]:
chords_counter = Counter()
for song in splits:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

29750

In [57]:
letters = list(string.ascii_uppercase)[:7]
non_chord_filter = [chord for chord in sorted_cc_df.index if chord[0] in letters]
cc_df = sorted_cc_df[sorted_cc_df.index.isin(non_chord_filter)]

# pd.set_option('display.max_rows', 1190)
## sorted_cc_df.head(70*17)
pd.set_option('display.max_rows', 100)
cc_df.iloc[400:500]

,chord_count
E7/B,478
A5+,478
Gb7M,478
Gm/D,474
Dbmaj7,474
B7sus4,470
Adim,467
Break:,464
A7sus,463
Ab9,463


In [ ]:
# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

## Creating List of Correctly Formatted Chords

In [59]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [60]:
chords_df = pd.DataFrame(index=['m', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', 'm-M7', '7', 'aug7', 'M7',  
                                'm9', '9', '7b9',  'M9', 'm11', '11',
                                'add2', 'add4', 'add6', 'add9', 
                                'madd2', 'madd4', 'madd6', 'madd9', 
                                '7add4', '7addb6', '7add6', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4'
                                ],
                         columns=major_chords)


In [61]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df.head(3)

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim
aug,Caug,C#aug,Dbaug,Daug,D#aug,Ebaug,Eaug,Faug,F#aug,Gbaug,Gaug,G#aug,Abaug,Aaug,A#aug,Bbaug,Baug


In [70]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 
                'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 
                'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']

slash_indexes = ['dim/b7', 'm/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/b7', 'm/7',
                'm7/b3', 'm7/4', 'm7/b5', 'm7/5', 'm7/7', 
                '/2', '/b3', '/3', '/4', '/5', '/6','/b7', '/7', 
                '7/b2', '7/2', '7/b3', '7/3', '7/4', '7/5', '7/b6', '7/6','7/7', 
                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/7',
                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/b7']

whole_to_half = {'b2': 1, '2': 2, 'b3': 3, '3': 4, '4': 5, 'b5': 6,
                 '5': 7, 'b6': 8,'6': 9, 'b7': 10, '7': 11}

In [76]:
# big function
def get_slash_notes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')
            h = whole_to_half[s[1]]
            sc = f'{chord}{s[0]}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if n in chord :
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
            elif len(chord) > 1:
                if '#' == chord[1]:
                    i = sharp_notes.index(chord[:2])
                    find_slash(chord, sharp_notes, i)
                elif 'b' == chord[1]:
                    i = flats_notes.index(chord[:2])
                    find_slash(chord, flats_notes, i)
    
    for chord in columns:
        sort_f_s(chord, sharp_notes)
        sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [79]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
slash_chords_df.shape

(80, 17)

In [80]:
80 * 17

1360

## Shove chords into correct format

#### For next time:
- merge chords
- delete everything other than those

In [85]:
substitutions = {'M7': ['7M', 'maj7'], '': 'maj', }

pandas.core.series.Series

In [102]:

def merge_chords(chords_column):
    merged_chords_column = chords_column.copy()
    
    for song in chords_column:
        print(row)
        for measure, chords in song.items():
            if 'bridge' in chords.lower():
                del song[measure]
            elif '%' in chords:
                ch_list = chords.split()
                for idx, ch in enumerate(ch_list):
                    if ch == '%':
                        ch.replace('%', ch_list[idx-1])
            song[measure] = chords.replace(substitutions)
        break
    
    return 'merged_chords_column'
        
merge_chords(split_raw_df.chords)

{3: 'G G/B C', 5: 'G', 7: 'G G/B C', 9: 'G', 13: 'G Em C G', 15: 'G Em C G', 19: 'G Em C G', 21: 'G Em C G', 23: 'G Em C G', 25: 'G Em C G', 27: 'G Em C G', 31: 'G G/B C', 33: 'G', 35: 'G G/B C', 37: 'G', 41: 'G Em C G', 43: 'G Em C G', 47: 'G Em C G', 49: 'G Em C G', 51: 'G Em C G', 53: 'G Em C G', 55: 'G Em C G', 57: 'G Em C G', 60: 'Bridge:', 61: 'Bm C', 63: 'Bm C', 69: 'G Em C G', 71: 'G Em C G', 73: 'G Em C G', 75: 'G Em C G', 77: 'G Em C G', 79: 'G Em C G', 84: 'G Em C', 86: 'C G G', 88: 'G Em C G'}
G G/B C
G
G G/B C
G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G G/B C
G
G G/B C
G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
Bridge:
Bm C
Bm C
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C G
G Em C
C G G
G Em C G


'merged_chords_column'